In [5]:
import pandas as pd
import numpy as np

In [256]:
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/DataCleaning-Audible'

In [2]:
%cd /content/gdrive/My Drive/DataCleaning-Audible

/content/gdrive/My Drive/DataCleaning-Audible


In [3]:
!ls

audible_cleaned.csv  audible_uncleaned.csv  kaggle.json


In [101]:
df = pd.read_csv('audible_uncleaned.csv')

In [124]:
#Look at df

In [102]:
df.shape

(87489, 8)

In [103]:
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [125]:
#start cleaning process by changing format to change dtypes

In [104]:
df['hour'] = 0
df['min'] = 0
df['hour'] = df.time.str.extract(r'^(\d+) hr').fillna(0)
df['min'] = df.time.str.extract(r'(\d+) min').fillna(0)
df = df.astype({'hour':'int', 'min':'int'})
df['hour'] = df['hour'] * 60

In [105]:
df = df.drop('time', axis = 1)
df['time'] = df['hour'] + df['min']

In [106]:
df = df.drop(['hour', 'min'], axis = 1)

In [107]:
df['time_in_minutes'] = df['time']
df = df.drop('time', axis = 1)

In [108]:
df['author'] = df['author'].str.replace('Writtenby:', '', regex = True)
df['author'] = df['author'].str.replace(r"([A-Z])", r" \1", regex = True)
df['narrator'] = df['narrator'].str.replace('Narratedby:', '', regex = True)
df['narrator'] = df['narrator'].str.replace(r"([A-Z])", r" \1", regex = True)

In [109]:
df['releasedate'] = pd.to_datetime(df['releasedate'])

In [110]:
df['language'] = df['language'].str.title()

In [111]:
df[['stars_5_scale', 'rating']] = df['stars'].str.split('stars', expand = True)

In [112]:
df['stars_5_scale'] = df['stars_5_scale'].str.replace(' out of 5', '')

In [113]:
df['rating'] = df['rating'].str.replace(' ratings| rating', '', regex = True)
df['rating'] = df['rating'].str.replace(',', '', regex = True)

In [114]:
not_rated = df[df['stars_5_scale'].str.contains('Not rated yet', case = False)].index
df.loc[not_rated,['stars_5_scale', 'rating']] = np.nan

In [115]:
df = df.astype({'stars_5_scale':'float', 'rating':'float'})
df = df.drop('stars', axis = 1)

In [116]:
df['price'] = df['price'].str.replace('free', '0', case = False, regex = True)
df['price'] = df['price'].str.replace(',', '', regex = True).astype(float)

In [119]:
df = df.rename(columns = {'name':'book_name', 'releasedate':'release_date'})

In [126]:
#rearrange columns for easier reading

In [121]:
df = df.reindex(columns = ['book_name', 'language', 'release_date', 'time_in_minutes', 'author', 'narrator', 'stars_5_scale', 'rating', 'price'])

In [127]:
#Completed cleaning

In [122]:
df.head()

,book_name,language,release_date,time_in_minutes,author,narrator,stars_5_scale,rating,price
0,Geronimo Stilton #11 & #12,English,2008-04-08,140,Geronimo Stilton,Bill Lobely,5.0,34.0,468.0
1,The Burning Maze,English,2018-01-05,788,Rick Riordan,Robbie Daymond,4.5,41.0,820.0
2,The Deep End,English,2020-06-11,123,Jeff Kinney,Dan Russell,4.5,38.0,410.0
3,Daughter of the Deep,English,2021-05-10,676,Rick Riordan,Soneela Nankani,4.5,12.0,615.0
4,"The Lightning Thief: Percy Jackson, Book 1",English,2010-01-13,600,Rick Riordan,Jesse Bernstein,4.5,181.0,820.0


In [123]:
df.dtypes

book_name                  object
language                   object
release_date       datetime64[ns]
time_in_minutes             int64
author                     object
narrator                   object
stars_5_scale             float64
rating                    float64
price                     float64
dtype: object